In [1]:
import pennylane as qml
import numpy as np
import sys

In [2]:
from pennylane_ls import *

In [3]:
from heroku_credentials import username, password

In [4]:
testDevice = FermionDevice(shots = 5, username = username, password = password)

In [5]:
testDevice.operations

{'ChemicalPotential',
 'HartreeFock',
 'Hop',
 'Inter',
 'Load',
 'OnSiteInteraction',
 'Phase',
 'Tunneling'}

### For the fermion simulator

In [6]:
testDevice.num_wires

8

In [7]:
@qml.qnode(testDevice)
def quantum_circuit(alpha=0):
    Load(wires=0)
    Load(wires=1)
    Load(wires=2)
    Hop(alpha,wires=[0,1,2,3])
    #Inter(alpha,wires=testDevice.wires)
    Inter(alpha,wires=[0,3])
    Phase(alpha, wires=[0, 2])
    return qml.sample(ParticleNumber(wires=testDevice.wires))

In [8]:
quantum_circuit(2,shots=10)

array([[1., 0., 1., 1., 0., 0., 0., 0.],
       [1., 0., 1., 1., 0., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0., 0., 0.],
       [1., 0., 1., 1., 0., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0., 0., 0.],
       [1., 0., 1., 1., 0., 0., 0., 0.],
       [1., 0., 1., 1., 0., 0., 0., 0.],
       [1., 0., 1., 1., 0., 0., 0., 0.],
       [1., 0., 1., 1., 0., 0., 0., 0.]])

In [9]:
print(quantum_circuit.draw())

 0: ──Load──╭Hop(2)──╭Inter(2)──╭Phase(2)──╭┤ Sample[ParticleNumber] 
 1: ──Load──├Hop(2)──│──────────│──────────├┤ Sample[ParticleNumber] 
 2: ──Load──├Hop(2)──│──────────╰Phase(2)──├┤ Sample[ParticleNumber] 
 3: ────────╰Hop(2)──╰Inter(2)─────────────├┤ Sample[ParticleNumber] 
 4: ───────────────────────────────────────├┤ Sample[ParticleNumber] 
 5: ───────────────────────────────────────├┤ Sample[ParticleNumber] 
 6: ───────────────────────────────────────├┤ Sample[ParticleNumber] 
 7: ───────────────────────────────────────╰┤ Sample[ParticleNumber] 



In [24]:
testDevice.expval(observable='ParticleNumber',wires=testDevice.wires)

3.0

In [25]:
testDevice.var(observable='ParticleNumber',wires=testDevice.wires)

0.4800000000000001

In [26]:
testDevice.probability(wires=testDevice.wires)

OrderedDict([((1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0), 0.6),
             ((1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.4)])

In [27]:
@qml.qnode(testDevice)
def simple_hopping(theta = 0):
    '''
    The circuit that simulates the experiments.
    
    theta ... angle of the hopping
    '''
    # load atoms
    FermionOps.Load(wires=0)
    FermionOps.Load(wires=1)
    
    # let them hop
    FermionOps.Hop(theta, wires=[0,1,2,3])

    # measure the occupation on the right side
    obs = FermionOps.ParticleNumber(0) @ FermionOps.ParticleNumber(1)
    return qml.expval(obs)

In [28]:
simple_hopping(2,shots=10)

TypeError: unhashable type: 'list'